# 01 — Exploratory Data Analysis: UCI Online Retail II

**Dataset:** UCI Online Retail II (id=502)  
**Source:** UK-based online gift retailer, transactions 01/12/2009 – 09/12/2011  
**Goal:** Profile the raw data, confirm key statistics, and understand purchase-hour distributions for the email send-time optimization pipeline.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Paths
import os
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
RAW_DIR = os.path.join(PROJECT_ROOT, 'data', 'raw')
CSV_PATH = os.path.join(RAW_DIR, 'online_retail_ii.csv')

print(f'Project root: {PROJECT_ROOT}')
print(f'CSV path: {CSV_PATH}')
print(f'File exists: {os.path.exists(CSV_PATH)}')


## 1. Load Raw Data


In [ ]:
df = pd.read_csv(CSV_PATH, parse_dates=['InvoiceDate'])
print(f'Total rows: {len(df):,}')
print(f'Columns: {df.columns.tolist()}')
print(f'Date range: {df["InvoiceDate"].min()} — {df["InvoiceDate"].max()}')
df.head()


## 2. Data Profile: Nulls and Duplicates


In [ ]:
print('=== NULL COUNTS ===')
null_counts = df.isnull().sum()
null_pct = (df.isnull().sum() / len(df) * 100).round(2)
null_summary = pd.DataFrame({'null_count': null_counts, 'null_pct': null_pct})
print(null_summary)

print(f'\n=== DUPLICATES ===')
n_dups = df.duplicated().sum()
print(f'Duplicate rows: {n_dups:,} ({n_dups/len(df)*100:.2f}%)')


## 3. Row Count Confirmation


In [ ]:
print(f'Total row count: {len(df):,}')
print(f'Expected ~1,067,371 — Match: {abs(len(df) - 1_067_371) < 100}')

# Sheet breakdown
yr_2009_2010 = df[df['InvoiceDate'].dt.year.isin([2009, 2010])]
yr_2010_2011 = df[df['InvoiceDate'].dt.year.isin([2010, 2011])]
print(f'\nRows by year:')
print(df['InvoiceDate'].dt.year.value_counts().sort_index())


## 4. Customer ID Analysis


In [ ]:
n_unique_customers = df['Customer ID'].nunique()
n_null_customers = df['Customer ID'].isna().sum()

print(f'Unique CustomerIDs (before filtering nulls): {n_unique_customers:,}')
print(f'Expected ~5,900 — in range: {5_700 < n_unique_customers < 6_100}')
print(f'Rows with null CustomerID: {n_null_customers:,} ({n_null_customers/len(df)*100:.1f}%)')

# After filtering
df_with_cust = df.dropna(subset=['Customer ID'])
print(f'\nRows after dropping null CustomerID: {len(df_with_cust):,}')
print(f'Unique CustomerIDs after filter: {df_with_cust["Customer ID"].nunique():,}')


## 5. Country Distribution


In [ ]:
uk_pct = (df['Country'] == 'United Kingdom').mean() * 100
print(f'UK row proportion: {uk_pct:.1f}%')
print(f'Expected ~90% UK — within range: {85 < uk_pct < 95}')

print('\n=== TOP 15 COUNTRIES BY TRANSACTION COUNT ===')
top_countries = df['Country'].value_counts().head(15)
print(top_countries)


In [ ]:
# Top countries by unique customer count
print('=== TOP 15 COUNTRIES BY UNIQUE CUSTOMER COUNT ===')
top_countries_cust = (
    df_with_cust.groupby('Country')['Customer ID']
    .nunique()
    .sort_values(ascending=False)
    .head(15)
)
print(top_countries_cust)

fig, ax = plt.subplots(figsize=(10, 5))
top_countries_cust.plot(kind='bar', ax=ax, color='steelblue', edgecolor='white')
ax.set_title('Top 15 Countries by Unique Customer Count', fontsize=14)
ax.set_xlabel('Country')
ax.set_ylabel('Unique Customers')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()


## 6. Cancellation Rate


In [ ]:
cancellations = df['Invoice'].astype(str).str.startswith('C')
cancel_rate = cancellations.mean() * 100
print(f'Cancellation rows: {cancellations.sum():,}')
print(f'Cancellation rate: {cancel_rate:.2f}%')

# By year
df['is_cancelled'] = cancellations
print('\nCancellations by year:')
print(df.groupby(df['InvoiceDate'].dt.year)['is_cancelled'].agg(['sum', 'mean'])
      .rename(columns={'sum': 'cancel_count', 'mean': 'cancel_rate'}))


## 7. Hour and Day-of-Week Distributions


In [ ]:
df['hour'] = df['InvoiceDate'].dt.hour
df['day_of_week'] = df['InvoiceDate'].dt.day_name()

print('=== TRANSACTION COUNT BY HOUR ===')
hour_counts = df.groupby('hour').size()
print(hour_counts)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Hour distribution
hour_counts.plot(kind='bar', ax=axes[0], color='steelblue', edgecolor='white')
axes[0].set_title('Transactions by Hour of Day', fontsize=13)
axes[0].set_xlabel('Hour')
axes[0].set_ylabel('Transaction Count')
axes[0].tick_params(axis='x', rotation=0)

# Day of week distribution
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_counts = df.groupby('day_of_week').size().reindex(dow_order)
dow_counts.plot(kind='bar', ax=axes[1], color='coral', edgecolor='white')
axes[1].set_title('Transactions by Day of Week', fontsize=13)
axes[1].set_xlabel('Day')
axes[1].set_ylabel('Transaction Count')
axes[1].tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show()


## 8. Modal Purchase Hour Distribution Across Customers


In [ ]:
# Compute each customer's modal (most common) purchase hour
# Re-derive df_with_cust now that 'hour' column has been added to df
df_with_cust_h = df.dropna(subset=['Customer ID'])

customer_modal_hour = (
    df_with_cust_h.groupby('Customer ID')['hour']
    .agg(lambda x: x.mode().iloc[0])
    .reset_index()
    .rename(columns={'hour': 'modal_hour'})
)

print(f'Customers with modal hour computed: {len(customer_modal_hour):,}')
print('\nModal hour distribution across customers:')
modal_dist = customer_modal_hour['modal_hour'].value_counts().sort_index()
print(modal_dist)

fig, ax = plt.subplots(figsize=(12, 5))
modal_dist.plot(kind='bar', ax=ax, color='mediumseagreen', edgecolor='white')
ax.set_title('Distribution of Modal Purchase Hour Across Customers', fontsize=14)
ax.set_xlabel('Hour of Day (modal purchase hour per customer)')
ax.set_ylabel('Number of Customers')
ax.tick_params(axis='x', rotation=0)
plt.tight_layout()
plt.show()

most_common_modal_hour = customer_modal_hour['modal_hour'].mode().iloc[0]
print(f'\nMost common modal purchase hour: {most_common_modal_hour}:00')


## 9. Summary Statistics


In [ ]:
print('=== EDA SUMMARY ===')
print(f'Total rows:                 {len(df):>12,}')
print(f'Unique CustomerIDs (raw):   {df["Customer ID"].nunique():>12,}')
print(f'Null CustomerID rows:       {df["Customer ID"].isna().sum():>12,}')
print(f'Duplicate rows:             {df.duplicated().sum():>12,}')
print(f'Cancellation rate:          {cancel_rate:>11.2f}%')
print(f'UK transaction proportion:  {uk_pct:>11.1f}%')
print(f'Date range start:           {df["InvoiceDate"].min()}')
print(f'Date range end:             {df["InvoiceDate"].max()}')
print(f'Peak transaction hour:      {hour_counts.idxmax()}:00')
print(f'Most common modal hour:     {most_common_modal_hour}:00')
print(f'Number of countries:        {df["Country"].nunique():>12,}')
